# Load

In [1]:
import numpy as np

In [156]:
dic_score = {"^": 3, "v": 1, ">": 0, "<": 2}

dic_moves = {">": (0, 1), "<": (0, -1), "^": (-1, 0), "v": (1, 0)}

dic_turn = {
    "R": {">":"v", "v": "<", "<": "^", "^": ">"},
    "L": {">":"^", "v": ">", "<": "v", "^": "<"},
    "I": {">": "<", "<": ">", "v": "^", "^": "v"}
}

In [190]:
with open("inputs/22", "r") as fp:
    data = fp.read()

size = 50

lines = data.split("\n")

puzzle = lines[:-2]
moves = lines[-2]


In [191]:
def parse_moves(moves):
    lst = []
    buff = []
    for c in moves:
        if c.isnumeric():
            buff.append(c)
        else:
            if len(buff) > 0:
                lst.append(int("".join(buff)))
                buff = []
            
            lst.append(c)
    
    if buff != []:
        lst.append(int("".join(buff)))

    return lst
                

In [193]:
def puzzle_to_map(puzzle):
    l0 = len(puzzle)
    l1 = max(list(map(len, puzzle)))
    mat = np.zeros((l0, l1))
    for idx, line in enumerate(puzzle):
        for jdx, c in enumerate(line):
            if c == " ":
                continue
            elif c == "#":
                mat[idx, jdx] = 2
            elif c == ".":
                mat[idx, jdx] = 1
    
    return mat.astype(int)

def get_blocks(mat, w=4):
    blks = []
    for i in range(mat.shape[0] // w):
        blk = []
        for j in range(mat.shape[1] // w):
            blk.append(mat[i*w:(i+1)*w, j*w:(j+1)*w])
        
        blks.append(blk)
    
    return blks

# Part 2


For the example:

```txt
  F
ABC
  DE
```

In [198]:
moves_parsed = parse_moves(moves)

mat = puzzle_to_map(puzzle)

blocks = get_blocks(mat, w=size)

### Example

In [130]:
# When moving, know if rotate the block or not.
dicNei = {
    "A": {"v": "Di", ">": "B", "<": "ER", "^": "Fi"},
    "B": {"v": "Dr", ">": "C", "<": "A", "^": "FR"},
    "C": {"v": "D", ">": "ER", "<": "B", "^": "F"},
    "D": {"v": "Ai", ">": "E", "<": "BR", "^": "C"},
    "E": {"v": "Ar", ">": "Fi", "<": "D", "^": "Cr"},
    "F": {"v": "C", ">": "Ei", "<": "Br", "^": "Ai"},
}

In [76]:
dic_assign = {
    "A": blocks[1][0],
    "B": blocks[1][1],
    "C": blocks[1][2],
    "D": blocks[2][2],
    "E": blocks[2][3],
    "F": blocks[0][2],
             }

size = len(blocks[0][0])

### My puzzle

```txt


 AB
 C
DE
F
```

In [211]:
dic_loc = {"A": (0, 1), "B": (0, 2), "C": (1, 1), "D": (2, 0), "E": (2, 1), "F": (3, 0)}

dic_assign = {}
for key, val in dic_loc.items():
    dic_assign[key] = blocks[val[0]][val[1]]


dicNei = {
    "A": {"v": "C", ">": "B", "<": "Di", "^": "FR"},
    "B": {"v": "CR", ">": "Ei", "<": "A", "^": "F"},
    "C": {"v": "E", ">": "Br", "<": "Dr", "^": "A"},
    "D": {"v": "F", ">": "E", "<": "Ai", "^": "CR"},
    "E": {"v": "FR", ">": "Bi", "<": "D", "^": "C"},
    "F": {"v": "B", ">": "Er", "<": "Ar", "^": "D"},
}

### Function

In [212]:
def change_cell(cell, pt, direction):
    # TODO: ensure all are positive
    
    # 1. Take the cell that is taken
    cell_next = dicNei[cell][direction]
    # 2. Rotate the point if necessary
    if len(cell_next) == 1: # one letter, no modification / turn
        pt1 = (pt[0], pt[1])
        dir1 = direction
        
    elif cell_next[1] == "i": # Inverse / Double rotation
        pt1 =  (size - pt[0]-1, size - pt[1]-1)
        dir1 = dic_turn["I"][direction]
        
    elif cell_next[1] == "r": # Rotate right, clockwiser
        # TODO
        pt1 = (size - pt[1] -1, pt[0])
        dir1 = dic_turn["L"][direction]
    
    else: # Rotate left, anticlockwise
        pt1 = (pt[1], size - pt[0] -1) # TO check
        dir1 = dic_turn["R"][direction]
        # TODO

    return (pt1[0] % size, pt1[1] % size), cell_next[0], dir1
    
        


def make_move(pt, direction, cell):
    """
    Given a position and a direction within a cell, suggest new location
    """
    d0, d1 = dic_moves[direction]
    pt1 = pt[0] + d0, pt[1] + d1
    if (pt1[0] == -1) | (pt1[1] == -1) | (pt1[0] == size) | (pt1[1] == size):
        # Condition to go into another cell
        return change_cell(cell, pt1, direction)

    else:
        # Stay in the current cell
        return pt1, cell, direction
    

def read_instruction(parsed_moves, verbose=True):
    """Main loop to execute the whole moves
    """
    pt = [0, 0] # Top left corner of the block
    cell = "A" # Change if necessary
    direction = ">"
    
    for instr in parsed_moves:
        if isinstance(instr, int):
            for _ in range(instr):
                # Try to see where the point would be at the next move
                pt1, cell1, dir1 = make_move(pt, direction, cell)
                
                if dic_assign[cell1][pt1[0], pt1[1]] == 1: # 1 == ., 2 == #
                    # If there is no stone, move to the new location
                    pt = pt1
                    cell = cell1
                    direction = dir1
                    if verbose:
                        print(pt, cell, direction)
                else:
                    if verbose:
                        # Otherwise, stay in place
                        print("# block in", pt1, "STOP")
                    break
                    
        
        else: # STR: R | C
            # Simple, just change the direction
            direction = dic_turn[instr][direction]
            if verbose:
                print("Rotate", instr, ":", direction)
            
    return pt, cell, direction

In [213]:
pt_end, cell_end, dir_end = read_instruction(moves_parsed, verbose=True)

(0, 1) A >
(0, 2) A >
(0, 3) A >
(0, 4) A >
(0, 5) A >
(0, 6) A >
(0, 7) A >
(0, 8) A >
(0, 9) A >
(0, 10) A >
(0, 11) A >
(0, 12) A >
(0, 13) A >
(0, 14) A >
(0, 15) A >
(0, 16) A >
Rotate L : ^
(16, 0) F >
(16, 1) F >
(16, 2) F >
(16, 3) F >
(16, 4) F >
(16, 5) F >
(16, 6) F >
(16, 7) F >
(16, 8) F >
(16, 9) F >
(16, 10) F >
(16, 11) F >
(16, 12) F >
(16, 13) F >
(16, 14) F >
(16, 15) F >
(16, 16) F >
(16, 17) F >
(16, 18) F >
(16, 19) F >
(16, 20) F >
(16, 21) F >
(16, 22) F >
(16, 23) F >
(16, 24) F >
(16, 25) F >
(16, 26) F >
(16, 27) F >
(16, 28) F >
(16, 29) F >
# block in (16, 30) STOP
Rotate L : ^
(15, 29) F ^
# block in (14, 29) STOP
Rotate R : >
(15, 30) F >
(15, 31) F >
(15, 32) F >
(15, 33) F >
(15, 34) F >
(15, 35) F >
(15, 36) F >
(15, 37) F >
# block in (15, 38) STOP
Rotate L : ^
(14, 37) F ^
(13, 37) F ^
(12, 37) F ^
(11, 37) F ^
(10, 37) F ^
(9, 37) F ^
(8, 37) F ^
(7, 37) F ^
(6, 37) F ^
(5, 37) F ^
(4, 37) F ^
(3, 37) F ^
(2, 37) F ^
(1, 37) F ^
(0, 37) F ^
(49, 37)

### Solution

```txt
 AB
 C
DE
F
```

In [215]:
1000 * (i0 * size + pt_end[0]+1) + 4*(j0 * size + pt_end[1]+1) + dic_score[dir_end]

129339